In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../../"))

from autocog.pm.sta.compile import Compiler
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
sources = ["""
prompt mmlu_main {
    is {
        topic is text<20>;
        question is text<50>;
        choices[4] is {
            value is text<40>;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}""","""
format bool {
    is enum("true","false");
    annotate "a bolean value is either true or false";
}

format sentence {
    argument length=50;
    argument language="english";
    is text<length=$length>;
    annotate f"a grammatically correct {language} sentence";
}

struct a_choice {
    is {
        value   is sentence<40>;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text<20>;
        question   is sentence;
        choices[4] is a_choice;
        answer     is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
"""]

In [4]:
for source in sources:
    compiler = Compiler(source=source)
    print(compiler.summarize())
    # print(compiler.program.prompts['mmlu_main'].fields[1].format.desc)
    # wrap_graphviz(compiler.program.toGraphViz())

> topic(text<length=20>): 
> question(text<length=50>): 
> choices(record)[4]: 
> > value(text<length=40>): 
> > correct(enum("yes","no")): 
> answer(repeat(choices.value)): 
--------------------

concrete=name='mmlu_main.question.sentence<length=50,language=english>' desc=['a grammatically correct english sentence'] refname='sentence<length=50,language=english>' length=50 within=None
concrete=name='mmlu_main.choices.value.sentence<length=40,language=english>' desc=['a grammatically correct english sentence'] refname='sentence<length=40,language=english>' length=40 within=None
concrete=name='mmlu_main.choices.correct.bool' desc=['a bolean value is either true or false'] refname='bool' values=['true', 'false']
> topic(text<length=20>): 
> question(sentence<length=50,language=english>): 
> choices(record)[4]: 
> > value(sentence<length=40,language=english>): 
> > correct(bool): 
> answer(repeat(choices.value)): 
--------------------
sentence<length=50,language=english>: a grammatically c